<h1 style="text-align: center;">Final Project </h1>

$\bullet$ __ We first download load the data __

In [1]:
import pandas as pd
import os

df_reviews = pd.DataFrame()
file_list = os.listdir("../reviews.json")

for file in file_list:
    # loading the reviews into a dataframe
    df = pd.read_json("../reviews.json/" + file, lines=True)
    df_reviews = pd.concat([df_reviews,df])

# removing rows containing NaN value and reaffecting index to the df
df_reviews = df_reviews.dropna() 
df_reviews = df_reviews.reset_index(drop=True)

In [2]:
# loading the business into a dataframe
df_business = pd.read_json("../yelp_dataset/yelp_academic_dataset_business.json",lines=True) #lines=True to load a file with more than one line of JSON

# removing rows containing NaN value and reaffecting index to the df
df_business = df_business.dropna() 
df_business = df_business.reset_index(drop=True)

In [3]:
print("Reviews keys : ", list(df_reviews.keys()))
print("Business keys : ", list(df_business.keys()))

Reviews keys :  ['business_id', 'date', 'review_id', 'stars', 'text', 'user_id']
Business keys :  ['address', 'attributes', 'business_id', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'name', 'neighborhood', 'postal_code', 'review_count', 'stars', 'state']


<hr>

$\bullet$ __ The idea is now to filter the business dataframe in order to select only the business that have reviews__

In [4]:
list_business = list(df_reviews.business_id.unique())

# selecting only the wanted business
df_business = df_business[df_business['business_id'].isin(list_business)]
df_business = df_business.reset_index(drop=True)

# strange thing : the two following lists do not have the same length
print(len(list_business) - len(df_business))
print(list(set(list_business) - set(list(df_business.business_id.values))))

12
['LHXisknIbUy_XtdEQc7x9w', '2NsEac9xCBI05bo5l4yI7Q', '-2ToCaDFpTNmmg3QFzxcWg', '8nmjvYX4N67cxOsGfbEgjg', 'v9u0NgjA4iQeKGevFnYSCA', 'of4V8nfW7GwJ03tLDdrOlA', 'hRK876bEBdPYAJKbg6pCEw', 'ho1vUk9R7Ole9aNmX6NMDA', 'X-b4-QvZLENnf3yFwhpSXQ', 'kamKFAkLzH-mXx5QmvCefg', 'a8ACgZ_bPPT6iRQ6R7Ridg', 'OFnDWBP8iXxgeBrfHA5fRA']


<hr>

# Analyze comment 

In [5]:
import nltk
from nltk import word_tokenize
labMT = pd.read_csv('data/s001.txt', sep="\t")

In [6]:
# function computing the sentimental value of a text
def computeSentiment(text):
    happiness_average, text = 0, word_tokenize(text)
    words = {word: text.count(word) for word in set(text)}
    
    for word in words:
        val_happiness = labMT[labMT["word"] == word].happiness_average.values
        # checking if the word is in labMT or not
        if len(val_happiness)>0:
            happiness_average += float(val_happiness[0]) * words[word] # taking into account the nb of occurences of the word
            
    return happiness_average / len(text)   

$\bullet$ __Now creating a dictionnary where keys are business id and value a list of all the comments related to this business. In order to compute the rating of each business. And another one where keys are business id and values the sentiment value__

In [18]:
dic_reviews = {business: [] for business in list_business}

for i in range(len(df_reviews)):
    comment = df_reviews.loc[i].text
    business = df_reviews.loc[i].business_id
    
    dic_reviews[business].append(comment)

In [58]:
from statistics import mean
dic_rate = {}

for elt in list(dic_reviews.keys())[:20]:
    dic_rate[elt] = mean([computeSentiment(text) for text in dic_reviews[elt]])

<hr>

# Showing positions on a map

In [28]:
# filtering business in order to get business in a small area
df_business["latitude"] = pd.to_numeric(df_business["latitude"]) # changing type of the column to float
df_business["longitude"] = pd.to_numeric(df_business["longitude"])

# df_business = df_business[(df_business["longitude"] > -90) & (df_business['longitude'] < -85) &(df_business['latitude'] < 42)]
# df_business = df_business.reset_index(drop=True)
# print(len(df_business))

Use either one of the following locations :
<li> First one gets all business locations </li>
<li> Second one gets only the locations of the business which sentimental mark is computed</li>

In [65]:
# creating a list containing all localisations of business
locations = []
for i in range(len(df_business)):
    locations.append((df_business.loc[i].latitude, df_business.loc[i].longitude))

562


In [70]:
# creating a list containing the localisations of all the analyzed business
locations = []
for business in list(dic_rate.keys()):
    locations.append((df_business[df_business['business_id'] == business].latitude.values[0], df_business[df_business['business_id'] == business].longitude.values[0]))

In [91]:
# here we compute the color of each marker (business) regarding their final mark
list_rates = list(dic_rate.values())

def aggregateColors(rate):
    x = int(255*(rate - min(list_rates))/(max(list_rates) - min(list_rates)))
    return (255 - x, x, 0, 0.5)

colors = [aggregateColors(elt) for elt in rate]

[(198, 57, 0, 0.5), (94, 161, 0, 0.5), (233, 22, 0, 0.5), (0, 255, 0, 0.5), (173, 82, 0, 0.5), (95, 160, 0, 0.5), (179, 76, 0, 0.5), (255, 0, 0, 0.5), (118, 137, 0, 0.5), (177, 78, 0, 0.5), (125, 130, 0, 0.5), (42, 213, 0, 0.5), (76, 179, 0, 0.5), (191, 64, 0, 0.5), (145, 110, 0, 0.5), (232, 23, 0, 0.5), (136, 119, 0, 0.5), (217, 38, 0, 0.5), (173, 82, 0, 0.5), (193, 62, 0, 0.5)]


In [92]:
import gmaps
gmaps.configure(api_key='API_KEY')

fig = gmaps.figure()
symbols = gmaps.symbol_layer(
        locations, fill_color=colors, stroke_color=colors)
fig.add_layer(symbols)

# hm = gmaps.heatmap_layer(locations, weights=rate, max_intensity=max(rate), point_radius=5.0)
# fig.add_layer(hm)

fig

Figure(layout=FigureLayout(height='420px'))